In [27]:
import requests
from bs4 import BeautifulSoup
import re
import os
 
ul = 'https://www.mbta.com/stops/subway'
response = requests.get(ul)
#response.encoding = 'gbk'  # 原始网页编码错误，utf-8也不管用，只能用gbk
html = response.text
# print(html)
soup = BeautifulSoup(html, 'lxml')  # 变成汤汁
 
 
def get_txt_name():  # 得到线路名称的前一步
    txt_src_name = []
  
    temp = soup.find_all('a', class_="c-call-to-action")
    print(temp)
    txt_src_name += temp
    return txt_src_name
   
#print(get_txt_name())  
def get_txtuseful_name():  # 得到可用的线路名称
    obj = []
    for each in get_txt_name():
        temp = re.findall(r'href="/schedules/(.+?)/line"', str(each))  # 从>匹配到<(不包含)，若要包含，则先使用re.compile，再search
        obj += temp
    return obj
print(get_txtuseful_name())  # 测试用


[<a class="c-call-to-action" href="/schedules/Red/line">              View Red Line info
</a>, <a class="c-call-to-action" href="/schedules/Mattapan/line">                  View Mattapan Trolley info
</a>, <a class="c-call-to-action" href="/schedules/Orange/line">              View Orange Line info
</a>, <a class="c-call-to-action" href="/schedules/Green/line">              View Green Line info
</a>, <a class="c-call-to-action" href="/schedules/Blue/line">              View Blue Line info
</a>]
['Red', 'Mattapan', 'Orange', 'Green', 'Blue']


['Alewife', 'Andrew', 'Ashmont', 'Braintree', 'Broadway', 'Central', 'Charles/MGH', 'Davis', 'Downtown Crossing', 'Fields Corner', 'Harvard', 'JFK/UMass', 'Kendall/MIT', 'North Quincy', 'Park Street', 'Porter', 'Quincy Adams', 'Quincy Center', 'Savin Hill', 'Shawmut', 'South Station', 'Wollaston', 'Ashmont', 'Butler', 'Capen Street', 'Cedar Grove', 'Central Avenue', 'Mattapan', 'Milton', 'Valley Road', 'Assembly', 'Back Bay', 'Chinatown', 'Community College', 'Downtown Crossing', 'Forest Hills', 'Green Street', 'Haymarket', 'Jackson Square', 'Malden Center', 'Massachusetts Avenue', 'North Station', 'Oak Grove', 'Roxbury Crossing', 'Ruggles', 'State', 'Stony Brook', 'Sullivan Square', 'Tufts Medical Center', 'Wellington', 'Allston Street', 'Arlington', 'Babcock Street', 'Back of the Hill', 'Beaconsfield', 'Blandford Street', 'Boston College', 'Boston University Central', 'Boston University East', 'Boston University West', 'Boylston', 'Brandon Hall', 'Brigham Circle', 'Brookline Hills', 

In [117]:
from bs4 import BeautifulSoup
import requests
import re
def get_lines_list(line_num):
    
 
        url = "https://www.mbta.com/schedules/"+line_num+"/line" 
        response = requests.get(url)
        
        soup = BeautifulSoup(response.content)
        origin_stop=[]
        temp1= soup.find_all('div', class_="route-branch-stop-name")
        origin_stop+=temp1
        #print(origin_stop)
        after_stop = []
        for each in origin_stop:
            temp2 = re.findall(r">(.+?)</a>", str(each))  # 从>匹配到<(不包含)，若要包含，则先使用re.compile，再search
            after_stop += temp2
        return after_stop
    


In [118]:
line=['Red','Blue','Orange','Mattapan','Green-B','Green-C','Green-D','Green-E']
for i in line:
    print(get_lines_list(i))

['Alewife', 'Davis', 'Porter', 'Harvard', 'Central', 'Kendall/\u200bMIT', 'Charles/\u200bMGH', 'Park Street', 'Downtown Crossing', 'South Station', 'Broadway', 'Andrew', 'JFK/\u200bUMass', 'North Quincy', 'Wollaston', 'Quincy Center', 'Quincy Adams', 'Braintree', 'Savin Hill', 'Fields Corner', 'Shawmut', 'Ashmont']
['Wonderland', 'Revere Beach', 'Beachmont', 'Suffolk Downs', 'Orient Heights', 'Wood Island', 'Airport', 'Maverick', 'Aquarium', 'State', 'Government Center', 'Bowdoin']
['Oak Grove', 'Malden Center', 'Wellington', 'Assembly', 'Sullivan Square', 'Community College', 'North Station', 'Haymarket', 'State', 'Downtown Crossing', 'Chinatown', 'Tufts Medical Center', 'Back Bay', 'Massachusetts Avenue', 'Ruggles', 'Roxbury Crossing', 'Jackson Square', 'Stony Brook', 'Green Street', 'Forest Hills']
['Ashmont', 'Cedar Grove', 'Butler', 'Milton', 'Central Avenue', 'Valley Road', 'Capen Street', 'Mattapan']
['Park Street', 'Boylston', 'Arlington', 'Copley', 'Hynes Convention Center', '

In [121]:
from collections import defaultdict
def build_connection(city_info):
    cities_connection = defaultdict(list)
    
    for j in line:
        cities = get_lines_list(j)
        for c1 in cities:
            for c2 in cities:
                if c1 == c2 : continue
                if abs(cities.index(c1)-cities.index(c2))==1:

                    cities_connection[c1].append(c2)
                    cities_connection[c1]=list(set(cities_connection[c1]))
    return cities_connection
stops=list(get_lines_list(i) for i in line)
stop_connection = build_connection(stops)

In [123]:
stop_connection

defaultdict(list,
            {'Alewife': ['Davis'],
             'Davis': ['Alewife', 'Porter'],
             'Porter': ['Davis', 'Harvard'],
             'Harvard': ['Central', 'Porter'],
             'Central': ['Harvard', 'Kendall/\u200bMIT'],
             'Kendall/\u200bMIT': ['Central', 'Charles/\u200bMGH'],
             'Charles/\u200bMGH': ['Park Street', 'Kendall/\u200bMIT'],
             'Park Street': ['Downtown Crossing',
              'Boylston',
              'Government Center',
              'Charles/\u200bMGH'],
             'Downtown Crossing': ['State',
              'Chinatown',
              'South Station',
              'Park Street'],
             'South Station': ['Downtown Crossing', 'Broadway'],
             'Broadway': ['Andrew', 'South Station'],
             'Andrew': ['JFK/\u200bUMass', 'Broadway'],
             'JFK/\u200bUMass': ['Andrew', 'North Quincy'],
             'North Quincy': ['JFK/\u200bUMass', 'Wollaston'],
             'Wollaston': ['North Q

In [129]:
def search(start,stop,connection_graph,sort_candidate):
    pathes = [[start]]
    visited = set()
    
    while pathes:
        path = pathes.pop()
        frontier = path[-1]
        if frontier in visited: continue
        successors = connection_graph[frontier]
        for city in successors:
            if city in path: continue
            new_path = path + [city]
            pathes.append(new_path)
            if city == stop: return new_path
        visited.add(frontier)
        pathes = sort_candidate(pathes)
#最长站点
def transfer_as_much(pathes):
    return sorted(pathes,key=len)
#最少站点
def transfer_as_less(pathes):
    return sorted(pathes,key=len,reverse=True)

In [128]:
from emoji import emojize
def pretty_print(cities):
    print(emojize(" :metro:-> ").join(cities))

In [130]:
pretty_print(search('Malden Center','Boston University Central',stop_connection,sort_candidate=transfer_as_less))

Malden Center 🚇-> Wellington 🚇-> Assembly 🚇-> Sullivan Square 🚇-> Community College 🚇-> North Station 🚇-> Haymarket 🚇-> Government Center 🚇-> Park Street 🚇-> Boylston 🚇-> Arlington 🚇-> Copley 🚇-> Hynes Convention Center 🚇-> Kenmore 🚇-> Blandford Street 🚇-> Boston University East 🚇-> Boston University Central


In [114]:
groups = [get_lines_list(i) for i in line]

indices = {}

for i, group in enumerate(groups):
    for stop in group:
        indices.setdefault(stop, set()).add(i)
print(groups)
print(indices)

[['Alewife', 'Davis', 'Porter', 'Harvard', 'Central', 'Kendall/\u200bMIT', 'Charles/\u200bMGH', 'Park Street', 'Downtown Crossing', 'South Station', 'Broadway', 'Andrew', 'JFK/\u200bUMass', 'North Quincy', 'Wollaston', 'Quincy Center', 'Quincy Adams', 'Braintree', 'Savin Hill', 'Fields Corner', 'Shawmut', 'Ashmont'], ['Wonderland', 'Revere Beach', 'Beachmont', 'Suffolk Downs', 'Orient Heights', 'Wood Island', 'Airport', 'Maverick', 'Aquarium', 'State', 'Government Center', 'Bowdoin'], ['Oak Grove', 'Malden Center', 'Wellington', 'Assembly', 'Sullivan Square', 'Community College', 'North Station', 'Haymarket', 'State', 'Downtown Crossing', 'Chinatown', 'Tufts Medical Center', 'Back Bay', 'Massachusetts Avenue', 'Ruggles', 'Roxbury Crossing', 'Jackson Square', 'Stony Brook', 'Green Street', 'Forest Hills'], ['Ashmont', 'Cedar Grove', 'Butler', 'Milton', 'Central Avenue', 'Valley Road', 'Capen Street', 'Mattapan'], ['Park Street', 'Boylston', 'Arlington', 'Copley', 'Hynes Convention Cente

In [86]:
def common_groups(stop1, stop2):
    return(indices.get(stop1, set()) is indices.get(stop2, set()))
    

In [87]:
print(common_groups('Alewife', 'Davis'))

False
